# Financial Analyst Agent for Annual Report Writing

In this demo, we introduce an agent that can analyze financial report.

In [1]:
# !pip install autogen

In [1]:
# %pip show pydantic

Name: pydantic
Version: 2.10.6
Summary: Data validation using Python type hints
Home-page: https://github.com/pydantic/pydantic
Author: 
Author-email: Samuel Colvin <s@muelcolvin.com>, Eric Jolibois <em.jolibois@gmail.com>, Hasan Ramezani <hasan.r67@gmail.com>, Adrian Garcia Badaracco <1755071+adriangb@users.noreply.github.com>, Terrence Dorsey <terry@pydantic.dev>, David Montague <david@pydantic.dev>, Serge Matveenko <lig@countzero.co>, Marcelo Trylesinski <marcelotryle@gmail.com>, Sydney Runkle <sydneymarierunkle@gmail.com>, David Hewitt <mail@davidhewitt.io>, Alex Hall <alex.mojaki@gmail.com>, Victorien Plot <contact@vctrn.dev>
License-Expression: MIT
Location: d:\anaconda3\envs\finrobot\lib\site-packages
Requires: annotated-types, pydantic-core, typing-extensions
Required-by: chromadb, fast-depends, fastapi, langchain, langchain-core, langsmith, marker-pdf, pdftext, pyautogen, pydantic-settings, surya-ocr, texify
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import autogen
from textwrap import dedent

In [2]:
import sys

sys.path.append(os.path.abspath(".."))  # Add the parent directory
from finrobot.utils import register_keys_from_json
from finrobot.agents.workflow import SingleAssistantShadow

After importing all the necessary packages and functions, we also need the config for OpenAI & SecApi & FMPApi here. 
- for openai configuration, rename OAI_CONFIG_LIST_sample to OAI_CONFIG_LIST and replace the api keys
- for Sec_api & FMP_api configuration, rename config_api_keys_sample to config_api_keys and replace the api keys

In [3]:
llm_config = {
    "config_list": autogen.config_list_from_json(
        "../OAI_CONFIG_LIST",
        filter_dict={
            "model": ["llama-3.3-70b-versatile"],
        },
    ),
    "timeout": 120,
    "temperature": 0.5,
}
register_keys_from_json("../config_api_keys")

In [3]:
# # ollama implementation

# config_list = [{
#     "model": "llama-3.3-70b-versatile",
#     "api_key": os.environ.get("GROQ_API_KEY"),
#     "api_type": "groq"
# }]
# register_keys_from_json("../config_api_keys")

In [5]:
# !pip install ag2[openai]

In [5]:
# Intermediate results will be saved in this directory
work_dir = "myreport"
os.makedirs(work_dir, exist_ok=True)

assistant = SingleAssistantShadow(
    "Expert_Investor",
    llm_config,
    max_consecutive_auto_reply=None,
    human_input_mode="TERMINATE",
)

In [6]:
company = "Apple Inc."
fyear = "2023"

message = dedent(
    f"""
    With the tools you've been provided, write an annual report based on {company}'s {fyear} 10-k report, format it into a pdf.
    Pay attention to the followings:
    - Explicitly explain your working plan before you kick off.
    - Use tools one by one for clarity, especially when asking for instructions. 
    - All your file operations should be done in "{work_dir}". 
    - Display any image in the chat once generated.
    - All the paragraphs should combine between 400 and 450 words, don't generate the pdf until this is explicitly fulfilled.
"""
)

assistant.chat(message, use_cache=True, max_turns=50,
               summary_method="last_msg")

User_Proxy (to Expert_Investor):


With the tools you've been provided, write an annual report based on Apple Inc.'s 2023 10-k report, format it into a pdf.
Pay attention to the followings:
- Explicitly explain your working plan before you kick off.
- Use tools one by one for clarity, especially when asking for instructions. 
- All your file operations should be done in "myreport". 
- Display any image in the chat once generated.
- All the paragraphs should combine between 400 and 450 words, don't generate the pdf until this is explicitly fulfilled.


--------------------------------------------------------------------------------
Expert_Investor (to User_Proxy):

***** Suggested tool call (call_za0m): get_sec_report *****
Arguments: 
{"ticker_symbol": "AAPL", "fyear": "2023"}
***********************************************************

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING FUNCTION get_sec_rep

D:\Anaconda3\envs\finrobot\lib\site-packages\autogen\oai\groq.py:279: UserWarning: Cost calculation not available for model llama-3.3-70b-versatile
  warnings.warn(f"Cost calculation not available for model {model}", UserWarning)


FMP api key found successfully.
User_Proxy (to Expert_Investor):

***** Response from calling tool (call_za0m) *****
Link: https://www.sec.gov/Archives/edgar/data/320193/000032019323000106/aapl-20230930.htm
Filing Date: 2023-11-03 00:00:00
**************************************************

--------------------------------------------------------------------------------
Expert_Investor (to User_Proxy):

***** Suggested tool call (call_ew0t): analyze_business_highlights *****
Arguments: 
{"ticker_symbol": "AAPL", "fyear": "2023", "save_path": "myreport/business_highlights.txt"}
************************************************************************
***** Suggested tool call (call_psht): analyze_income_stmt *****
Arguments: 
{"ticker_symbol": "AAPL", "fyear": "2023", "save_path": "myreport/income_stmt_analysis.txt"}
****************************************************************
***** Suggested tool call (call_66p5): analyze_segment_stmt *****
Arguments: 
{"ticker_symbol": "AAPL", "fye

FileNotFoundError: [Errno 2] No such file or directory: "myreport/business_highlights.txt\n\nmyreport/income_stmt_analysis.txt\n\nmyreport/segment_analysis.txt\n\nmyreport/risk_assessment.txt\n\nmyreport/competitors_analysis.txt\n\nmyreport/income_summarization.txt\n\n{'Rating': 'buy', 'Target Price': 'Failed to retrieve data: 403', '6m avg daily vol (USDmn)': '57.84', 'Closing Price (USD)': '176.28', 'Market Cap (USDmn)': '2739799.63', '52 Week Price Range (USD)': '126.30 - 193.60', 'BVPS (USD)': '3.95'}\n\npe performance chart saved to <img myreport/pe_eps_performance.png>\n\nlast year stock performance chart saved to <img myreport/share_performance.png>"

The Rest Cells are optional, simply used to show the generated PDF

In [7]:
!pip install PyMuPDF

   ---------------------------------------- 0.0/16.5 MB ? eta -:--:--
    --------------------------------------- 0.3/16.5 MB ? eta -:--:--
   - -------------------------------------- 0.5/16.5 MB 4.2 MB/s eta 0:00:04
   --- ------------------------------------ 1.3/16.5 MB 2.7 MB/s eta 0:00:06
   ---- ----------------------------------- 1.8/16.5 MB 2.6 MB/s eta 0:00:06
   ----- ---------------------------------- 2.4/16.5 MB 2.7 MB/s eta 0:00:06
   ------- -------------------------------- 3.1/16.5 MB 2.9 MB/s eta 0:00:05
   --------- ------------------------------ 3.9/16.5 MB 3.0 MB/s eta 0:00:05
   ----------- ---------------------------- 4.7/16.5 MB 3.1 MB/s eta 0:00:04
   ------------- -------------------------- 5.8/16.5 MB 3.4 MB/s eta 0:00:04
   ---------------- ----------------------- 6.8/16.5 MB 3.5 MB/s eta 0:00:03
   ------------------- -------------------- 8.1/16.5 MB 3.8 MB/s eta 0:00:03
   ------------------------ --------------- 10.0/16.5 MB 4.2 MB/s eta 0:00:02
   ---------

In [8]:
import io
import fitz
from PIL import Image


pdf = fitz.open("../report/Apple_Annual_Report_2023.pdf")
page = pdf.load_page(0)
pix = page.get_pixmap()

# Convert the Pixmap to a PIL Image
img = Image.open(io.BytesIO(pix.tobytes("png")))
display(img)

FileNotFoundError: no such file: '../report/Apple_Annual_Report_2023.pdf'